In [83]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [84]:
# poem generation
# Deep learning: a combination of VAE/GAN
# condition the generator of the GAN for rubustness
# latent space!!
# how do you evaluate these?
# discriminator

# test using a model on text and poems to predict whether a given text is a poem or not.
# maybe no input!
# my goal will be to generate 10,000 poems


# future project would be to write books using ai. minimum of 10-50 pages based on the most popular topic on xyz site?
# based on stars guess where the picture was taken in space?

In [85]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging
import re
import contractions
import nltk

In [86]:
df = pd.read_pickle('mostly_english_poems.pickle')
print(len(df))

26817


In [87]:
poems = df.content

In [88]:
def remove_punc(dirty_line) -> str:
    cleaner = dirty_line.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    cleaned_line = re.sub('\s\s+' , ' ', cleaner)
    return cleaned_line

In [89]:
def expand_contractions(text):
    expanded_words = []   
    for word in text.split():
        # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))  
    
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [90]:
def clean_poems(list_of_words) -> list[str]:
    cleaned_poems = []
    for string in list_of_words:

        lines = string.split('\n')
        good_lines = ''

        for line in lines:
            # replaces contractions with their expanded version: can't -> cannot
            removed_contractions = expand_contractions(line)
            # removes punctuation from the text and makes words lowercase
            cleaned_line = remove_punc(removed_contractions).lower()
            good_lines += cleaned_line + ' NEWLINE '

        # removes the ' NEWLINE ' from the end of the poem
        cleaned_poems.append(good_lines[:len(good_lines)-9])
        
    return cleaned_poems

In [ ]:
# this pipeline is shit!!!
# replace \n with newline
# expand contractions
# split into tokens
# convert into lowecase
# Decoding Unicode characters into a normalized form, such as UTF8???
# remove numbers???
# remove all punctuation
# filter out non alphabetic
# recontruct into poems
# us isalpha
# string = re.sub(r"[,.”;@—#“…?!&$-’_)–(*)]+", ' ', string)
# string = re.sub('\s\s+' , ' ', cleaner)

In [91]:
poems = clean_poems(poems)

In [92]:
for i in range(len(poems)):
    if 'translated by ' in poems[i]:
        print(poems[i])

translated by NEWLINE roderick gill NEWLINE o faint remembrances of vanished days NEWLINE that stole away on such a velvet wing NEWLINE over meads and groves over plains and mountain ways  NEWLINE what grief and sorrow to my heart you bring  NEWLINE come back without the shadow of your care  NEWLINE come back in silence and without a moan  NEWLINE as the birds cross the unregarding air NEWLINE till none may tell the whence or whither flown  NEWLINE come back amid the pallor of the moon NEWLINE that silvers all the azure rifts at sea  NEWLINE or in the deadly mist that in a swoon NEWLINE engulfs afar the green palm’s royal tree  NEWLINE bring back the murmur of the doves that made NEWLINE their little nests so neighborly to mine  NEWLINE the vibrant airs––the fragrances that played NEWLINE around the peaks that saw my cradle shine  NEWLINE sing in my ear the melodies of old  NEWLINE so sweet and joyous to my inmost heart  NEWLINE o faint remembrances two breasts should hold  NEWLINE two

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oliver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
Poems = df.content.map(lambda x: nltk.tokenize.word_tokenize(x))

In [18]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)